In [ ]:
%pip install transformers
%pip install datasets
%pip install huggingface_hub
%pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 wh

In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import torch
from transformers import DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# Load your dataset
df = pd.read_excel('/content/intent_classification_train.xlsx')

intent_label_dict = {0: 'Divorce', 1: 'Consumer protection',2: 'Inheritance', 3: 'Copyright', 4: 'Other'}

df.replace({'Divorce':0, 'Consumer protection':1, 'Inheritance':2, 'Copyright':3, 'Other':4}, inplace = True)

X = df['Query'].copy()
y = df['Intent'].copy()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y)

train_df = pd.concat([X_train, y_train], axis = 1)
test_df = pd.concat([X_test, y_test], axis = 1)

# Convert to Hugging Face Dataset format
raw_datasets = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "eval": Dataset.from_pandas(test_df)
})

<ipython-input-3-d20f2cc7ebf7>:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({'Divorce':0, 'Consumer protection':1, 'Inheritance':2, 'Copyright':3, 'Other':4}, inplace = True)


In [ ]:
train_df['Intent'].value_counts()

,count
Intent,
2,40
3,40
4,40
0,40
1,39


In [ ]:
test_df['Intent'].value_counts()

,count
Intent,
0,10
4,10
2,10
1,10
3,10


In [ ]:
checkpoint = "distilbert-base-uncased"
classifier = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5) # Get the classifier
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenized_datasets = raw_datasets.map(lambda dataset: tokenizer(dataset['Query'], truncation=True), batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/199 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Query', 'Intent', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 199
    })
    eval: Dataset({
        features: ['Query', 'Intent', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 50
    })
})


In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["Query", "__index_level_0__"])
tokenized_datasets = tokenized_datasets.rename_column("Intent", "labels")
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 199
    })
    eval: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 50
    })
})


In [ ]:
# Padding for batch of data that will be fed into model for training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training args
training_args = TrainingArguments("test-trainer", num_train_epochs=5, evaluation_strategy="epoch",
                                  weight_decay=5e-4, save_strategy="no", report_to="none")

# Define a compute_metrics function for multi-class classification
def compute_metrics(p):
    # Extract the predictions
    preds = p.predictions.argmax(-1)  # Get the index of the maximum logit value for each example
    labels = p.label_ids

    # Compute accuracy
    accuracy = accuracy_score(labels, preds)

    # Compute precision, recall, f1 for each class (weighted average)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Define trainer
trainer = Trainer(
    classifier,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["eval"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-b2027d09a8eb>:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.711686,0.940000,0.941818,0.940000,0.939950
2,No log,0.179094,0.960000,0.963636,0.960000,0.959900
3,No log,0.089560,0.960000,0.963636,0.960000,0.959900
4,No log,0.076246,0.960000,0.963636,0.960000,0.959900
5,No log,0.069788,0.960000,0.963636,0.960000,0.959900


TrainOutput(global_step=125, training_loss=0.3451881408691406, metrics={'train_runtime': 284.3933, 'train_samples_per_second': 3.499, 'train_steps_per_second': 0.44, 'total_flos': 3679789329060.0, 'train_loss': 0.3451881408691406, 'epoch': 5.0})

In [ ]:
metrics_train = trainer.evaluate(eval_dataset=tokenized_datasets["train"])
print("train Set Metrics:", metrics_train)

train Set Metrics: {'eval_loss': 0.019582338631153107, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 5.046, 'eval_samples_per_second': 39.437, 'eval_steps_per_second': 4.954, 'epoch': 5.0}


In [ ]:
metrics_test = trainer.evaluate(eval_dataset=tokenized_datasets["eval"])
print("Test Set Metrics:", metrics_test)

Test Set Metrics: {'eval_loss': 0.06978783011436462, 'eval_accuracy': 0.96, 'eval_precision': 0.9636363636363636, 'eval_recall': 0.96, 'eval_f1': 0.9598997493734336, 'eval_runtime': 1.4641, 'eval_samples_per_second': 34.151, 'eval_steps_per_second': 4.781, 'epoch': 5.0}


In [ ]:
# Example input text
new_text = "I just want divorce!"

# Tokenize the new text (same as during training)
inputs = tokenizer(new_text, return_tensors="pt", padding='max_length', truncation=True, max_length=128)

# Convert tokenized input into a Dataset
predict_dataset = Dataset.from_dict({
    'input_ids': inputs['input_ids'],
    'attention_mask': inputs['attention_mask']
})

# Get predictions from the trainer
predictions = trainer.predict(predict_dataset)

# Extract the logits
logits = predictions.predictions

# Get the index of the highest logit (the predicted class)
predicted_label = torch.argmax(torch.tensor(logits), dim=1).item()

print(f"Predicted intent: {intent_label_dict[predicted_label]}")

Predicted intent: Divorce


In [ ]:
# Example input text
new_text = "Hope I get rich someday"

# Tokenize the new text (same as during training)
inputs = tokenizer(new_text, return_tensors="pt", padding='max_length', truncation=True, max_length=128)

# Convert tokenized input into a Dataset
predict_dataset = Dataset.from_dict({
    'input_ids': inputs['input_ids'],
    'attention_mask': inputs['attention_mask']
})

# Get predictions from the trainer
predictions = trainer.predict(predict_dataset)

# Extract the logits
logits = predictions.predictions

# Get the index of the highest logit (the predicted class)
predicted_label = torch.argmax(torch.tensor(logits), dim=1).item()

print(f"Predicted intent: {intent_label_dict[predicted_label]}")

Predicted intent: Other


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [ ]:
# Save the model and tokenizer locally
trainer.save_model('content/intent_classifier')
tokenizer.save_pretrained('content/intent_tokenizer')

('content/intent_tokenizer/tokenizer_config.json',
 'content/intent_tokenizer/special_tokens_map.json',
 'content/intent_tokenizer/vocab.txt',
 'content/intent_tokenizer/added_tokens.json',
 'content/intent_tokenizer/tokenizer.json')

In [ ]:
# Load the saved model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained('content/intent_classifier')
tokenizer = AutoTokenizer.from_pretrained('content/intent_tokenizer')

# Push to Hugging Face Hub
model.push_to_hub("Sachin891/LegalRAG-intent-classification-distilbert")
tokenizer.push_to_hub("Sachin891/LegalRAG-intent-classification-tokenizer")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sachin891/LegalRAG-intent-classification-tokenizer/commit/95f61fba5e9aa9eda6f87e72a863bdcd78f30c44', commit_message='Upload tokenizer', commit_description='', oid='95f61fba5e9aa9eda6f87e72a863bdcd78f30c44', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Sachin891/LegalRAG-intent-classification-tokenizer', endpoint='https://huggingface.co', repo_type='model', repo_id='Sachin891/LegalRAG-intent-classification-tokenizer'), pr_revision=None, pr_num=None)